# SWB analysis pipeline part 4: 

Freq-Band Averaged Time Courses - Single ROI

*Created: 07/14/24* \
*Updated: XX/XX/2024*


In [62]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib as mpl
from matplotlib import cm



import warnings
warnings.filterwarnings('ignore')




In [2]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

08012024


In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import analysis_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *

In [4]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
# subj_ids


In [5]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


In [6]:
### load ROI reref master 
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


# Get data 

In [9]:
freqs = {'hfa':np.array([70,200]),'gamma':np.array([30,70]),'beta':np.array([13,30]),
         'alpha':np.array([9,13]),'theta':np.array([4,9]),'delta':np.array([2,4])}
band_rois = {'hfa':['pins','ofc','motor','dlpfc','hpc','dmpfc'],
             'gamma':['dlpfc','motor','pins','hpc','ofc','acc','ains','amy'],
             'beta':['motor','hpc','ains','vlpfc','acc','ofc'],
             'alpha':['ains','amy','dlpfc','hpc','motor','pins'],
             'theta':['acc','amy','hpc','ains','motor','pins']}

In [21]:
roi_df = roi_reref_labels_master_df[roi_reref_labels_master_df.roi.isin(plot_roi_list)]
roi_df['unique_reref_ch'] = roi_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
# roi_subjs  = roi_df.subj_id.unique().tolist()
# roi_elecs  = {f'{subj}':roi_df[roi_df.subj_id == subj].unique_reref_ch.tolist() for subj in roi_subjs}

# roi_elecs  = {f'{subj}':roi_df[roi_df.subj_id == subj].reref_ch_names.tolist() for subj in roi_subjs}
# roi_ch_df = 
# roi_elecs

In [63]:
roi_df

,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi,unique_reref_ch
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc,MS002_lacas1-lacas2
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc,MS002_lacas2-lacas3
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc,MS002_lacas3-lacas4
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc,MS002_lacas4-lacas5
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc,MS002_lacas5-lacas6
...,...,...,...,...,...,...,...,...,...,...,...
1855,DA039,rof5-rof6,rof5,anode,right frontal orbital 3 d,27.012676,40.339247,3.532617,r,ofc,DA039_rof5-rof6
1857,DA039,rof7-rof8,rof7,anode,right anterior pars triangularis a,30.972853,40.382590,9.957380,r,vlpfc,DA039_rof7-rof8
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc,DA039_rsgcc6-rsgcc7
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc,DA039_rsgcc7-rsgcc8


In [ ]:
### plot_band = str id of band to plot
plot_band  = 'beta'
### plot_roi = roi str according to roi_reref_labels_master
# plot_roi   = 'dmpfc'
plot_roi_list = band_rois[plot_band]
### freq_range = np array of upper/lower freq bounds of plot_band - either manual or subset from freqs above
# freq_range = np.array([9,12])
freq_range = freqs[plot_band]

In [64]:
freq_range

array([13, 30])

In [72]:
# roi_elecs  = {f'{subj}':roi_df[roi_df.subj_id == subj].reref_ch_names.tolist() for subj in roi_subjs}
roi_ch_df = roi_df.copy()[['subj_id','reref_ch_names','unique_reref_ch','roi']]
roi_ch_df

,subj_id,reref_ch_names,unique_reref_ch,roi
0,MS002,lacas1-lacas2,MS002_lacas1-lacas2,acc
1,MS002,lacas2-lacas3,MS002_lacas2-lacas3,acc
2,MS002,lacas3-lacas4,MS002_lacas3-lacas4,acc
3,MS002,lacas4-lacas5,MS002_lacas4-lacas5,acc
4,MS002,lacas5-lacas6,MS002_lacas5-lacas6,acc
...,...,...,...,...
1855,DA039,rof5-rof6,DA039_rof5-rof6,ofc
1857,DA039,rof7-rof8,DA039_rof7-rof8,vlpfc
1877,DA039,rsgcc6-rsgcc7,DA039_rsgcc6-rsgcc7,vlpfc
1878,DA039,rsgcc7-rsgcc8,DA039_rsgcc7-rsgcc8,vlpfc


In [73]:
roi_ch_df['bdi'] = np.nan
bdi_col_vals = np.concatenate([np.repeat(subj_info_df[subj_info_df.subj_id==subj_id].bdi.values,
                                         len(roi_ch_df[roi_ch_df.subj_id==subj_id])) 
                               for subj_id in subj_ids],axis=0)
roi_ch_df['bdi'] = bdi_col_vals

In [74]:
roi_ch_df

,subj_id,reref_ch_names,unique_reref_ch,roi,bdi
0,MS002,lacas1-lacas2,MS002_lacas1-lacas2,acc,14
1,MS002,lacas2-lacas3,MS002_lacas2-lacas3,acc,14
2,MS002,lacas3-lacas4,MS002_lacas3-lacas4,acc,14
3,MS002,lacas4-lacas5,MS002_lacas4-lacas5,acc,14
4,MS002,lacas5-lacas6,MS002_lacas5-lacas6,acc,14
...,...,...,...,...,...
1855,DA039,rof5-rof6,DA039_rof5-rof6,ofc,22
1857,DA039,rof7-rof8,DA039_rof7-rof8,vlpfc,22
1877,DA039,rsgcc6-rsgcc7,DA039_rsgcc6-rsgcc7,vlpfc,22
1878,DA039,rsgcc7-rsgcc8,DA039_rsgcc7-rsgcc8,vlpfc,22


In [ ]:
roi_band_pow_df

In [71]:
subj_elecs

['MS002_lacas1-lacas2',
 'MS002_lacas2-lacas3',
 'MS002_lacas3-lacas4',
 'MS002_lacas4-lacas5',
 'MS002_lacas5-lacas6',
 'MS002_laims1-laims2',
 'MS002_laims2-laims3',
 'MS002_laims3-laims4',
 'MS002_laims4-laims5',
 'MS002_laims5-laims6',
 'MS002_laims6-laims7',
 'MS002_lhplt1-lhplt2',
 'MS002_lhplt2-lhplt3',
 'MS002_lhplt3-lhplt4',
 'MS002_lhplt4-lhplt5',
 'MS002_lloif1-lloif2',
 'MS002_lloif2-lloif3',
 'MS002_lloif3-lloif4',
 'MS002_lloif6-lloif7',
 'MS002_lloif7-lloif8',
 'MS002_lloif8-lloif9',
 'MS002_lmoif2-lmoif3',
 'MS002_lmoif8-lmoif9',
 'MS002_lmoif9-lmoif10',
 'MS002_lmoif10-lmoif11',
 'MS002_lmoif11-lmoif12',
 'MS002_lmoif12-lmoif13',
 'MS002_lsif3-lsif4',
 'MS002_lsif4-lsif5',
 'MS002_lsif5-lsif6',
 'MS002_lsif6-lsif7',
 'MS002_lsif7-lsif8',
 'MS002_racas1-racas2',
 'MS002_racas2-racas3',
 'MS002_racas3-racas4',
 'MS002_raims1-raims2',
 'MS002_raims2-raims3',
 'MS002_raims3-raims4',
 'MS002_raims6-raims7',
 'MS002_rhplt1-rhplt2',
 'MS002_rhplt2-rhplt3',
 'MS002_rhplt3-rhpl

In [80]:
drop_cols = ['RT', 'CpeOnset',
       'GambleChoice', 'TrialType', 'SafeBet', 'LowBet', 'HighBet', 'GambleEV',
       'Outcome', 'Profit', 'TotalProfit', 'CR', 'choiceEV', 'RPE','decisionCF']

In [85]:
subj_elecs = roi_ch_df[roi_ch_df.subj_id == 'MS002']
power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
roi_epochs   = power_epochs.copy().pick_channels(subj_elecs.reref_ch_names.unique().tolist())
del power_epochs

roi_pow_df   = roi_epochs.to_data_frame()

roi_band_df  = roi_pow_df[(roi_pow_df.freq >= freq_range[0])&(roi_pow_df.freq <= freq_range[1])].groupby(
              ['epoch','time']).agg('mean').reset_index()

roi_band_df  = roi_band_df.melt(id_vars=['epoch','time','freq'],
                            value_vars=subj_elecs.reref_ch_names.unique().tolist(),
                            var_name='reref_ch_names', value_name='band_pow',
                            ignore_index = False).set_index('epoch')

roi_metadata = roi_epochs.metadata.copy()   
roi_metadata['epoch'] = roi_metadata.Round-1

    # merge pow and metadata dfs keeping epoch as index 
band_timeseries_df = pd.merge(roi_band_df, roi_metadata,left_on=['epoch'],
                            right_on=['epoch']).reset_index(drop=True).set_index('epoch')

del roi_epochs, roi_pow_df, roi_band_df, roi_metadata

band_timeseries_df  = band_timeseries_df[band_timeseries_df.RT>0.3].reset_index(drop=True)
band_timeseries_df  = band_timeseries_df[band_timeseries_df.Round != 76].reset_index(drop=True)
band_timeseries_df  = band_timeseries_df[band_timeseries_df.decisionCPE != 0]

band_timeseries_df  = band_timeseries_df.drop(['RT', 'CpeOnset',
                       'GambleChoice', 'TrialType', 'SafeBet', 'LowBet', 'HighBet', 'GambleEV',
                       'Outcome', 'Profit', 'TotalProfit', 'CR', 'choiceEV', 'RPE','decisionCF'],axis=1)

band_timeseries_df['cpe']       = band_timeseries_df.decisionCPE.copy()
band_timeseries_df['cpe_split'] = band_timeseries_df['cpe'].apply(lambda x: 0 if x < 0 else 1)
band_timeseries_df['cpe']       = norm_zscore(band_timeseries_df['cpe'].values)
band_timeseries_df['epoch_num'] = band_timeseries_df.index # keep epoch as a column
band_timeseries_df['subj_id']   = subj_id
band_timeseries_df['roi']       = subj_elecs[subj_elecs.subj_id == subj_id].roi.unique()[0]
band_timeseries_df['bdi']       = subj_elecs[subj_elecs.subj_id == subj_id].bdi.values[0]

#     band_timeseries_df['TrialEV']         = band_timeseries_df.GambleEV - roi_regression_df.SafeBet
#     band_timeseries_df['GambleChoice']    = band_timeseries_df['GambleChoice'].astype('category')
#     band_timeseries_df['TrialType']       = band_timeseries_df['TrialType'].astype('category')


band_timeseries_df['unique_reref_ch'] = band_timeseries_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)

    
band_timeseries_df

Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns


,time,freq,reref_ch_names,band_pow,Round,decisionCPE,decisionRegret,decisionRelief,cpe,cpe_split,epoch_num,subj_id,roi,bdi,unique_reref_ch
0,0.000,20.743342,lacas1-lacas2,-0.639674,1,-0.57,-0.57,0.0,-0.397504,0,0,MS002,acc,14,MS002_lacas1-lacas2
1,0.002,20.743342,lacas1-lacas2,-0.636421,1,-0.57,-0.57,0.0,-0.397504,0,1,MS002,acc,14,MS002_lacas1-lacas2
2,0.004,20.743342,lacas1-lacas2,-0.632104,1,-0.57,-0.57,0.0,-0.397504,0,2,MS002,acc,14,MS002_lacas1-lacas2
3,0.006,20.743342,lacas1-lacas2,-0.626684,1,-0.57,-0.57,0.0,-0.397504,0,3,MS002,acc,14,MS002_lacas1-lacas2
4,0.008,20.743342,lacas1-lacas2,-0.620113,1,-0.57,-0.57,0.0,-0.397504,0,4,MS002,acc,14,MS002_lacas1-lacas2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11107395,2.992,20.743342,rmoif9-rmoif10,0.341398,150,1.10,0.00,1.1,0.620917,1,11107395,MS002,acc,14,MS002_rmoif9-rmoif10
11107396,2.994,20.743342,rmoif9-rmoif10,0.360611,150,1.10,0.00,1.1,0.620917,1,11107396,MS002,acc,14,MS002_rmoif9-rmoif10
11107397,2.996,20.743342,rmoif9-rmoif10,0.376315,150,1.10,0.00,1.1,0.620917,1,11107397,MS002,acc,14,MS002_rmoif9-rmoif10
11107398,2.998,20.743342,rmoif9-rmoif10,0.388261,150,1.10,0.00,1.1,0.620917,1,11107398,MS002,acc,14,MS002_rmoif9-rmoif10


In [84]:
band_timeseries_df.drop(['RT', 'CpeOnset',
       'GambleChoice', 'TrialType', 'SafeBet', 'LowBet', 'HighBet', 'GambleEV',
       'Outcome', 'Profit', 'TotalProfit', 'CR', 'choiceEV', 'RPE','decisionCF'],axis=1)

,time,freq,reref_ch_names,band_pow,Round,decisionCPE,decisionRegret,decisionRelief,cpe,cpe_split,epoch_num,subj_id,roi,bdi,unique_reref_ch
0,0.000,20.743342,lacas1-lacas2,-0.639674,1,-0.57,-0.57,0.0,-0.397504,0,0,MS002,acc,14,MS002_lacas1-lacas2
1,0.002,20.743342,lacas1-lacas2,-0.636421,1,-0.57,-0.57,0.0,-0.397504,0,1,MS002,acc,14,MS002_lacas1-lacas2
2,0.004,20.743342,lacas1-lacas2,-0.632104,1,-0.57,-0.57,0.0,-0.397504,0,2,MS002,acc,14,MS002_lacas1-lacas2
3,0.006,20.743342,lacas1-lacas2,-0.626684,1,-0.57,-0.57,0.0,-0.397504,0,3,MS002,acc,14,MS002_lacas1-lacas2
4,0.008,20.743342,lacas1-lacas2,-0.620113,1,-0.57,-0.57,0.0,-0.397504,0,4,MS002,acc,14,MS002_lacas1-lacas2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11107395,2.992,20.743342,rmoif9-rmoif10,0.341398,150,1.10,0.00,1.1,0.620917,1,11107395,MS002,acc,14,MS002_rmoif9-rmoif10
11107396,2.994,20.743342,rmoif9-rmoif10,0.360611,150,1.10,0.00,1.1,0.620917,1,11107396,MS002,acc,14,MS002_rmoif9-rmoif10
11107397,2.996,20.743342,rmoif9-rmoif10,0.376315,150,1.10,0.00,1.1,0.620917,1,11107397,MS002,acc,14,MS002_rmoif9-rmoif10
11107398,2.998,20.743342,rmoif9-rmoif10,0.388261,150,1.10,0.00,1.1,0.620917,1,11107398,MS002,acc,14,MS002_rmoif9-rmoif10


In [ ]:
drop_cols = ['RT', 'CpeOnset',
       'GambleChoice', 'TrialType', 'SafeBet', 'LowBet', 'HighBet', 'GambleEV',
       'Outcome', 'Profit', 'TotalProfit', 'CR', 'choiceEV', 'RPE','decisionCF']

In [86]:

roi_band_pow_df = []

for subj_id in subj_ids:
      
    subj_elecs = roi_ch_df[roi_ch_df.subj_id == subj_id]
    power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
    roi_epochs   = power_epochs.copy().pick_channels(subj_elecs.reref_ch_names.unique().tolist())
    del power_epochs

    roi_pow_df   = roi_epochs.to_data_frame()

    roi_band_df  = roi_pow_df[(roi_pow_df.freq >= freq_range[0])&(roi_pow_df.freq <= freq_range[1])].groupby(
                  ['epoch','time']).agg('mean').reset_index()

    roi_band_df  = roi_band_df.melt(id_vars=['epoch','time','freq'],
                                value_vars=subj_elecs.reref_ch_names.unique().tolist(),
                                var_name='reref_ch_names', value_name='band_pow',
                                ignore_index = False).set_index('epoch')

    roi_metadata = roi_epochs.metadata.copy()   
    roi_metadata['epoch'] = roi_metadata.Round-1

        # merge pow and metadata dfs keeping epoch as index 
    band_timeseries_df = pd.merge(roi_band_df, roi_metadata,left_on=['epoch'],
                                right_on=['epoch']).reset_index(drop=True).set_index('epoch')

    del roi_epochs, roi_pow_df, roi_band_df, roi_metadata

    band_timeseries_df  = band_timeseries_df[band_timeseries_df.RT>0.3].reset_index(drop=True)
    band_timeseries_df  = band_timeseries_df[band_timeseries_df.Round != 76].reset_index(drop=True)
    band_timeseries_df  = band_timeseries_df[band_timeseries_df.decisionCPE != 0]

    band_timeseries_df  = band_timeseries_df.drop(['RT', 'CpeOnset',
                           'GambleChoice', 'TrialType', 'SafeBet', 'LowBet', 'HighBet', 'GambleEV',
                           'Outcome', 'Profit', 'TotalProfit', 'CR', 'choiceEV', 'RPE','decisionCF'],axis=1)

    band_timeseries_df['cpe']       = band_timeseries_df.decisionCPE.copy()
    band_timeseries_df['cpe_split'] = band_timeseries_df['cpe'].apply(lambda x: 0 if x < 0 else 1)
    band_timeseries_df['cpe']       = norm_zscore(band_timeseries_df['cpe'].values)
    band_timeseries_df['epoch_num'] = band_timeseries_df.index # keep epoch as a column
    band_timeseries_df['subj_id']   = subj_id
    band_timeseries_df['roi']       = subj_elecs[subj_elecs.subj_id == subj_id].roi.unique()[0]
    band_timeseries_df['bdi']       = subj_elecs[subj_elecs.subj_id == subj_id].bdi.values[0]

    #     band_timeseries_df['TrialEV']         = band_timeseries_df.GambleEV - roi_regression_df.SafeBet
    #     band_timeseries_df['GambleChoice']    = band_timeseries_df['GambleChoice'].astype('category')
    #     band_timeseries_df['TrialType']       = band_timeseries_df['TrialType'].astype('category')


    band_timeseries_df['unique_reref_ch'] = band_timeseries_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)


    roi_band_pow_df.append(band_timeseries_df)


Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/MS011_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Adding metadata with 19 

In [88]:

# roi_band_pow_df = pd.concat(roi_band_pow_df).reset_index()
roi_band_pow_df['bdi_split'] = roi_band_pow_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

roi_band_pow_df

,index,time,freq,reref_ch_names,band_pow,Round,decisionCPE,decisionRegret,decisionRelief,cpe,cpe_split,epoch_num,subj_id,roi,bdi,unique_reref_ch,bdi_split
0,0,0.000,20.743342,lacas1-lacas2,-0.639674,1,-0.57,-0.57,0.0,-0.397504,0,0,MS002,acc,14,MS002_lacas1-lacas2,0
1,1,0.002,20.743342,lacas1-lacas2,-0.636421,1,-0.57,-0.57,0.0,-0.397504,0,1,MS002,acc,14,MS002_lacas1-lacas2,0
2,2,0.004,20.743342,lacas1-lacas2,-0.632104,1,-0.57,-0.57,0.0,-0.397504,0,2,MS002,acc,14,MS002_lacas1-lacas2,0
3,3,0.006,20.743342,lacas1-lacas2,-0.626684,1,-0.57,-0.57,0.0,-0.397504,0,3,MS002,acc,14,MS002_lacas1-lacas2,0
4,4,0.008,20.743342,lacas1-lacas2,-0.620113,1,-0.57,-0.57,0.0,-0.397504,0,4,MS002,acc,14,MS002_lacas1-lacas2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152028780,5876410,2.992,20.743342,rvmot6-rvmot7,2.943456,150,-2.00,-2.00,0.0,-1.231942,0,5876410,DA039,ains,22,DA039_rvmot6-rvmot7,1
152028781,5876411,2.994,20.743342,rvmot6-rvmot7,3.313729,150,-2.00,-2.00,0.0,-1.231942,0,5876411,DA039,ains,22,DA039_rvmot6-rvmot7,1
152028782,5876412,2.996,20.743342,rvmot6-rvmot7,3.700957,150,-2.00,-2.00,0.0,-1.231942,0,5876412,DA039,ains,22,DA039_rvmot6-rvmot7,1
152028783,5876413,2.998,20.743342,rvmot6-rvmot7,4.103089,150,-2.00,-2.00,0.0,-1.231942,0,5876413,DA039,ains,22,DA039_rvmot6-rvmot7,1


In [35]:
# # roi_ch_df.drop_duplicates('unique_reref_ch')
# subj_elecs

#     metadata_df          = roi_metadata[subj_id].reset_index()
#     metadata_df['epoch'] = metadata_df.Round-1
    
#     roi_df      = roi_band_pow[subj_id]
#     roi_df      = roi_df.set_index('epoch')
    
#     # merge pow and metadata dfs keeping epoch as index 
#     roi_regression_df = pd.merge(roi_df, metadata_df,left_on=['epoch'],
#                                 right_on=['epoch']).reset_index(drop=True).set_index('epoch')
    
#     roi_regression_df  = roi_regression_df[roi_regression_df.RT>0.3].reset_index(drop=True)
#     roi_regression_df  = roi_regression_df[roi_regression_df.Round != 76].reset_index(drop=True)
#     roi_regression_df  = roi_regression_df[roi_regression_df.decisionCPE != 0]
    
#     roi_regression_df['cpe']             = roi_regression_df.decisionCPE.copy()
#     roi_regression_df['cpe_split']       = roi_regression_df['cpe'].apply(lambda x: 0 if x < 0 else 1)
#     roi_regression_df['TrialEV']         = roi_regression_df.GambleEV - roi_regression_df.SafeBet
#     roi_regression_df['GambleChoice']    = roi_regression_df['GambleChoice'].astype('category')
#     roi_regression_df['TrialType']       = roi_regression_df['TrialType'].astype('category')
#     roi_regression_df['epoch_num']       = roi_regression_df.index # keep epoch as a column
#     roi_regression_df['subj_id']         = subj_id
#     roi_regression_df['roi']             = plot_roi
#     roi_regression_df['bdi']             = subj_info_df[subj_info_df.subj_id == subj_id].bdi.values[0]
#     roi_regression_df['unique_reref_ch'] = roi_regression_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
    
    
# #         # zscore continuous regressors AFTER removing bad trials!! 
#     for reg in continuous_regressors:
#         roi_regression_df[reg] = norm_zscore(roi_regression_df[reg].values) # zscore is /2std

#     plot_dfs.append(roi_regression_df)
#       roi_ch_df

In [ ]:
# roi_band_pow = {}
# roi_metadata = {}
# band_all_roi_subj = {}


# for roi in plot_roi_list:

#     roi_info = roi_ch_df[roi_ch_df.roi==roi]
    
#     for subj_id in roi_subjs:
#         subj_elecs = roi_info[roi_info.subj_id == subj_id].unique_reref_ch.tolist()
        
        
#         # load cpe power epochs for single subject 
#         power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
#         roi_epochs   = power_epochs.copy().pick_channels(roi_elecs[subj_id])
#         del power_epochs

#         roi_pow_df  = roi_epochs.to_data_frame()
#         roi_band_df = roi_pow_df[(roi_pow_df.freq >= freq_range[0])&(roi_pow_df.freq <= freq_range[1])].groupby(
#                       ['epoch','time']).agg('mean').reset_index()

#         roi_band_df  = roi_band_df.melt(id_vars=['epoch','time','freq'],
#                                         value_vars=roi_elecs[subj_id],
#                                         var_name='reref_ch_names', value_name='band_pow',
#                                         ignore_index = False)

#         roi_band_pow[subj_id] = roi_band_df
#         roi_metadata[subj_id] = roi_epochs.metadata.copy()   
#         del roi_epochs, roi_pow_df, roi_band_df 

     

    

In [ ]:
# ##### regressors to zscore for plot data 
# continuous_regressors = ['decisionCPE','TrialEV','TotalProfit','decisionRegret','decisionRelief','RPE','Profit']


In [ ]:
# plot_dfs = []

# ### update metadata - zscore continuous regressors + add RT info
# for subj_id in roi_metadata.keys():
#     metadata_df          = roi_metadata[subj_id].reset_index()
#     metadata_df['epoch'] = metadata_df.Round-1
    
#     roi_df      = roi_band_pow[subj_id]
#     roi_df      = roi_df.set_index('epoch')
    
#     # merge pow and metadata dfs keeping epoch as index 
#     roi_regression_df = pd.merge(roi_df, metadata_df,left_on=['epoch'],
#                                 right_on=['epoch']).reset_index(drop=True).set_index('epoch')
    
#     roi_regression_df  = roi_regression_df[roi_regression_df.RT>0.3].reset_index(drop=True)
#     roi_regression_df  = roi_regression_df[roi_regression_df.Round != 76].reset_index(drop=True)
#     roi_regression_df  = roi_regression_df[roi_regression_df.decisionCPE != 0]
    
#     roi_regression_df['cpe']             = roi_regression_df.decisionCPE.copy()
#     roi_regression_df['cpe_split']       = roi_regression_df['cpe'].apply(lambda x: 0 if x < 0 else 1)
#     roi_regression_df['TrialEV']         = roi_regression_df.GambleEV - roi_regression_df.SafeBet
#     roi_regression_df['GambleChoice']    = roi_regression_df['GambleChoice'].astype('category')
#     roi_regression_df['TrialType']       = roi_regression_df['TrialType'].astype('category')
#     roi_regression_df['epoch_num']       = roi_regression_df.index # keep epoch as a column
#     roi_regression_df['subj_id']         = subj_id
#     roi_regression_df['roi']             = plot_roi
#     roi_regression_df['bdi']             = subj_info_df[subj_info_df.subj_id == subj_id].bdi.values[0]
#     roi_regression_df['unique_reref_ch'] = roi_regression_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
    
    
# #         # zscore continuous regressors AFTER removing bad trials!! 
#     for reg in continuous_regressors:
#         roi_regression_df[reg] = norm_zscore(roi_regression_df[reg].values) # zscore is /2std

#     plot_dfs.append(roi_regression_df)
      

In [ ]:
# roi_plot_df = pd.concat(plot_dfs).reset_index()

In [ ]:
# roi_plot_df 

# Start plotting

In [ ]:
sns.set_context('poster')
sns.set_style('ticks')



In [ ]:
ylim = [np.min(roi_plot_df.band_pow),np.max(roi_plot_df.band_pow)]

In [89]:

bdi_data    = roi_band_pow_df[roi_band_pow_df.bdi_split == 0].reset_index(drop=True) 
bdi_data

,index,time,freq,reref_ch_names,band_pow,Round,decisionCPE,decisionRegret,decisionRelief,cpe,cpe_split,epoch_num,subj_id,roi,bdi,unique_reref_ch,bdi_split
0,0,0.000,20.743342,lacas1-lacas2,-0.639674,1,-0.57,-0.57,0.0,-0.397504,0,0,MS002,acc,14,MS002_lacas1-lacas2,0
1,1,0.002,20.743342,lacas1-lacas2,-0.636421,1,-0.57,-0.57,0.0,-0.397504,0,1,MS002,acc,14,MS002_lacas1-lacas2,0
2,2,0.004,20.743342,lacas1-lacas2,-0.632104,1,-0.57,-0.57,0.0,-0.397504,0,2,MS002,acc,14,MS002_lacas1-lacas2,0
3,3,0.006,20.743342,lacas1-lacas2,-0.626684,1,-0.57,-0.57,0.0,-0.397504,0,3,MS002,acc,14,MS002_lacas1-lacas2,0
4,4,0.008,20.743342,lacas1-lacas2,-0.620113,1,-0.57,-0.57,0.0,-0.397504,0,4,MS002,acc,14,MS002_lacas1-lacas2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112309318,1073210,2.992,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073210,DA037,hpc,4,DA037_rof3-rof4,0
112309319,1073211,2.994,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073211,DA037,hpc,4,DA037_rof3-rof4,0
112309320,1073212,2.996,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073212,DA037,hpc,4,DA037_rof3-rof4,0
112309321,1073213,2.998,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073213,DA037,hpc,4,DA037_rof3-rof4,0


In [95]:
bdi_data

,index,time,freq,reref_ch_names,band_pow,Round,decisionCPE,decisionRegret,decisionRelief,cpe,cpe_split,epoch_num,subj_id,roi,bdi,unique_reref_ch,bdi_split,new_pow
0,0,0.000,20.743342,lacas1-lacas2,-0.639674,1,-0.57,-0.57,0.0,-0.397504,0,0,MS002,acc,14,MS002_lacas1-lacas2,0,NaN
1,1,0.002,20.743342,lacas1-lacas2,-0.636421,1,-0.57,-0.57,0.0,-0.397504,0,1,MS002,acc,14,MS002_lacas1-lacas2,0,NaN
2,2,0.004,20.743342,lacas1-lacas2,-0.632104,1,-0.57,-0.57,0.0,-0.397504,0,2,MS002,acc,14,MS002_lacas1-lacas2,0,NaN
3,3,0.006,20.743342,lacas1-lacas2,-0.626684,1,-0.57,-0.57,0.0,-0.397504,0,3,MS002,acc,14,MS002_lacas1-lacas2,0,NaN
4,4,0.008,20.743342,lacas1-lacas2,-0.620113,1,-0.57,-0.57,0.0,-0.397504,0,4,MS002,acc,14,MS002_lacas1-lacas2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112309318,1073210,2.992,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073210,DA037,hpc,4,DA037_rof3-rof4,0,NaN
112309319,1073211,2.994,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073211,DA037,hpc,4,DA037_rof3-rof4,0,NaN
112309320,1073212,2.996,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073212,DA037,hpc,4,DA037_rof3-rof4,0,NaN
112309321,1073213,2.998,20.743342,rof3-rof4,NaN,150,1.50,0.00,1.5,0.894221,1,1073213,DA037,hpc,4,DA037_rof3-rof4,0,NaN


In [97]:
for roi in bdi_data.roi.unique().tolist():
    roi_data = bdi_data[bdi_data.roi == roi]

NameError: name 'plot_roi' is not defined

In [ ]:
roi_bdi_data = 

In [ ]:
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']
bdi_keys   = ['Low BDI','High BDI']

# bdi_data['new_pow'] = np.nan
# if bdi_thresh == 0:
#     bdi = 'low'
# else:
#     bdi = 'high'

for roi in bdi_data.roi.unique().tolist():
    
    roi_data = bdi_data[bdi_data.roi == roi]
    roi_data['new_pow'] = np.nan
    
    for elec in roi_data.unique_reref_ch.unique().tolist():
        elec_df = roi_data[roi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        roi_data['new_pow'][roi_data.unique_reref_ch == elec] = elec_df['power']


    power_avg = roi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}_all_{plot_roi}.pdf'

    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)

    ymin = rp.round(np.min(power_avg),3)
    ymax = np.round(np.max(power_avg),3)
    print(ymin,ymax)
# plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)

# # Plot the time resolved power with SEM shading
# for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

#     subset = power_avg[power_avg['cpe_split'] == cpe_split]

#     plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])

#     ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
#             framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

#     plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
#          subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)

# plt.ylim([])
# plt.tight_layout()
# plt.title(f'{elec}')
# plt.xlabel(r'Time (ms)')
# plt.ylabel(fr'${plot_band}_{{zpow}}$')
# sns.despine()
    

In [94]:
roi_plot_df.bdi_split.unique()

NameError: name 'roi_plot_df' is not defined

In [ ]:
##### plot by subj 

fig_dir   = f'{base_dir}ephys_analysis/figs/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']


for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}_all_{plot_roi}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)

    plt.ylim([])
    plt.tight_layout()
        plt.title(f'{elec}')
    plt.xlabel(r'Time (ms)')
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()
    
#     plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
#     bbox_inches='tight', pad_inches=0.1,
#     facecolor='auto', edgecolor='auto',
#     backend=None)
    del bdi_data
